In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import to_categorical
from sklearn.preprocessing import OneHotEncoder

In [2]:
def CategoricalConverter(column):
    onehotencoder = OneHotEncoder()
    prod = onehotencoder.fit_transform(pd.DataFrame(column)).toarray() # Convertir a categórico
    prod = pd.DataFrame(prod, columns = np.sort(column.unique())) #Hacer un df con columnas
    return prod

In [3]:
trs = pd.read_csv('../Bases/Transaccion_train.csv')
trs.drop(['Unnamed: 0' , 'Id_Producto' , 'Tipo' , 'Fecha'] , axis = 1 , inplace = True)

In [4]:
trs['Signo'] = trs['Signo'].fillna("Positivo")

In [5]:
trsfinal = trs.copy()
trsfinal.set_index('id' , inplace = True)
trsfinal.dropna(inplace = True) #se eliminaron 1.439.192 filas

In [6]:
trsfinal

,Producto-Tipo,Signo,Monto,Periodo
id,,,,
1,F-D,Positivo,377142.843750,201901
1,F-D,Positivo,577142.875000,201903
1,F-D,Positivo,548571.437500,201904
1,F-D,Positivo,291428.562500,201905
1,F-D,Positivo,405714.281250,201906
...,...,...,...,...
79539,G-K,Negativo,196444.718750,202006
79539,G-K,Negativo,128644.289062,202006
79539,G-K,Negativo,124086.859375,202006


# Filtrar Productos que Necesito

In [7]:
AA = trsfinal[trsfinal['Producto-Tipo'] == 'A-A']
AA.reset_index(inplace = True)
BB = trsfinal[trsfinal['Producto-Tipo'] == 'B-B']
BB.reset_index(inplace = True)
CD = trsfinal[trsfinal['Producto-Tipo'] == 'C-D']
CD.reset_index(inplace = True)
DE = trsfinal[trsfinal['Producto-Tipo'] == 'D-E']
DE.reset_index(inplace = True)
EE = trsfinal[trsfinal['Producto-Tipo'] == 'E-E']
EE.reset_index(inplace = True)

In [8]:
targetdf = pd.concat([AA,BB,CD,DE,EE])
targetdf

,id,Producto-Tipo,Signo,Monto,Periodo
0,3,A-A,Positivo,234285.71875,201906
1,4,A-A,Positivo,362857.15625,201912
2,6,A-A,Positivo,220000.00000,201901
3,6,A-A,Positivo,220000.00000,201904
4,7,A-A,Positivo,220000.00000,201903
...,...,...,...,...,...
67529,79537,E-E,Positivo,124385.00000,201907
67530,79537,E-E,Positivo,124392.00000,201905
67531,79537,E-E,Positivo,124590.00000,201901
67532,79539,E-E,Positivo,120714.00000,201911


# Eliminar del Df Original

In [9]:
trsfinal.reset_index(inplace = True)
trsfinal.set_index('Producto-Tipo' , inplace = True)
trsfinal.drop(['A-A' , 'B-B' , 'C-D' , 'D-E' ,'E-E'] , inplace = True)
trsfinal.reset_index(inplace = True)
trsfinal

,Producto-Tipo,id,Signo,Monto,Periodo
0,F-D,1,Positivo,377142.843750,201901
1,F-D,1,Positivo,577142.875000,201903
2,F-D,1,Positivo,548571.437500,201904
3,F-D,1,Positivo,291428.562500,201905
4,F-D,1,Positivo,405714.281250,201906
...,...,...,...,...,...
23857901,G-K,79539,Negativo,196444.718750,202006
23857902,G-K,79539,Negativo,128644.289062,202006
23857903,G-K,79539,Negativo,124086.859375,202006
23857904,A-G,79539,Positivo,185393.000000,201912


In [10]:
signo = trsfinal['Signo'].values
montoSigno = []
for i in signo:
    if i == 'Positivo':
        montoSigno.append(1)
    else:
        montoSigno.append(-1)
montoSigno = np.array(montoSigno)
monto = trsfinal['Monto'].values
trsfinal['MontoSigno'] = np.multiply(monto,montoSigno) # Multiplicación
trsfinal.drop(['Signo' , 'Monto'] , axis = 1 , inplace = True)
trsfinal.reset_index(drop = True , inplace = True)

In [11]:
trsfinal

,Producto-Tipo,id,Periodo,MontoSigno
0,F-D,1,201901,377142.843750
1,F-D,1,201903,577142.875000
2,F-D,1,201904,548571.437500
3,F-D,1,201905,291428.562500
4,F-D,1,201906,405714.281250
...,...,...,...,...
23857901,G-K,79539,202006,-196444.718750
23857902,G-K,79539,202006,-128644.289062
23857903,G-K,79539,202006,-124086.859375
23857904,A-G,79539,201912,185393.000000


# Pivot Table

In [13]:
pivot2 = pd.pivot_table(trsfinal , values = 'MontoSigno' , index = ['id' , 'Periodo'] , columns = 'Producto-Tipo' , fill_value = 0 , aggfunc = np.sum)

In [14]:
pivot2.reset_index(inplace = True)

In [15]:
pivot2

Producto-Tipo,id,Periodo,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
0,1,201901,0,-2.946055e+06,0.0,0.000000,0,0,377142.84375,-377142.84375,0.0,377142.84375,-6.108959e+05
1,1,201902,260048,-2.888311e+06,0.0,0.000000,0,0,0.00000,-120000.00000,0.0,120000.00000,-4.875886e+05
2,1,201903,281602,-3.674344e+06,0.0,0.000000,0,0,577142.87500,-577142.87500,0.0,577142.87500,-7.334999e+05
3,1,201904,274541,-3.200335e+06,0.0,0.000000,0,0,548571.43750,-548571.43750,0.0,548571.43750,-6.080734e+05
4,1,201905,459132,-3.104311e+06,0.0,0.000000,0,0,291428.56250,-291428.56250,0.0,291428.56250,-2.447571e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409148,79539,202003,0,0.000000e+00,0.0,-191898.421875,0,0,0.00000,0.00000,0.0,120000.00000,-7.975236e+05
1409149,79539,202004,0,0.000000e+00,0.0,-192012.421875,0,0,0.00000,0.00000,0.0,120000.00000,-8.189417e+05
1409150,79539,202005,0,0.000000e+00,0.0,-550620.312500,0,0,0.00000,0.00000,0.0,120000.00000,-7.462784e+05
1409151,79539,202006,0,0.000000e+00,0.0,-192155.718750,0,-120357,0.00000,0.00000,0.0,120000.00000,-1.204000e+06


In [16]:
pivot3 = pivot2.copy()

In [17]:
pivot3id = pivot3['id']
pivot3.drop(['id'] , axis = 1 , inplace = True)

In [18]:
pivot3

Producto-Tipo,Periodo,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
0,201901,0,-2.946055e+06,0.0,0.000000,0,0,377142.84375,-377142.84375,0.0,377142.84375,-6.108959e+05
1,201902,260048,-2.888311e+06,0.0,0.000000,0,0,0.00000,-120000.00000,0.0,120000.00000,-4.875886e+05
2,201903,281602,-3.674344e+06,0.0,0.000000,0,0,577142.87500,-577142.87500,0.0,577142.87500,-7.334999e+05
3,201904,274541,-3.200335e+06,0.0,0.000000,0,0,548571.43750,-548571.43750,0.0,548571.43750,-6.080734e+05
4,201905,459132,-3.104311e+06,0.0,0.000000,0,0,291428.56250,-291428.56250,0.0,291428.56250,-2.447571e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
1409148,202003,0,0.000000e+00,0.0,-191898.421875,0,0,0.00000,0.00000,0.0,120000.00000,-7.975236e+05
1409149,202004,0,0.000000e+00,0.0,-192012.421875,0,0,0.00000,0.00000,0.0,120000.00000,-8.189417e+05
1409150,202005,0,0.000000e+00,0.0,-550620.312500,0,0,0.00000,0.00000,0.0,120000.00000,-7.462784e+05
1409151,202006,0,0.000000e+00,0.0,-192155.718750,0,-120357,0.00000,0.00000,0.0,120000.00000,-1.204000e+06


In [19]:
def test(x):
    if x>0:
        x = 1
    else:
        x = 0
    return x

In [20]:
#for i in pivot3.columns.values:
    #pivot3[i] = pivot3[i].apply(test)
pivot3['id'] = pivot3id

In [21]:
pivot3

Producto-Tipo,Periodo,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K,id
0,201901,0,-2.946055e+06,0.0,0.000000,0,0,377142.84375,-377142.84375,0.0,377142.84375,-6.108959e+05,1
1,201902,260048,-2.888311e+06,0.0,0.000000,0,0,0.00000,-120000.00000,0.0,120000.00000,-4.875886e+05,1
2,201903,281602,-3.674344e+06,0.0,0.000000,0,0,577142.87500,-577142.87500,0.0,577142.87500,-7.334999e+05,1
3,201904,274541,-3.200335e+06,0.0,0.000000,0,0,548571.43750,-548571.43750,0.0,548571.43750,-6.080734e+05,1
4,201905,459132,-3.104311e+06,0.0,0.000000,0,0,291428.56250,-291428.56250,0.0,291428.56250,-2.447571e+05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409148,202003,0,0.000000e+00,0.0,-191898.421875,0,0,0.00000,0.00000,0.0,120000.00000,-7.975236e+05,79539
1409149,202004,0,0.000000e+00,0.0,-192012.421875,0,0,0.00000,0.00000,0.0,120000.00000,-8.189417e+05,79539
1409150,202005,0,0.000000e+00,0.0,-550620.312500,0,0,0.00000,0.00000,0.0,120000.00000,-7.462784e+05,79539
1409151,202006,0,0.000000e+00,0.0,-192155.718750,0,-120357,0.00000,0.00000,0.0,120000.00000,-1.204000e+06,79539


In [22]:
#pivot3.to_pickle('BinaryTransaction.pkl')

In [23]:
pivotFinal = pivot3.copy()

In [24]:
pivotFinal.set_index('id' , inplace = True)

In [25]:
pivotFinal

Producto-Tipo,Periodo,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K
id,,,,,,,,,,,,
1,201901,0,-2.946055e+06,0.0,0.000000,0,0,377142.84375,-377142.84375,0.0,377142.84375,-6.108959e+05
1,201902,260048,-2.888311e+06,0.0,0.000000,0,0,0.00000,-120000.00000,0.0,120000.00000,-4.875886e+05
1,201903,281602,-3.674344e+06,0.0,0.000000,0,0,577142.87500,-577142.87500,0.0,577142.87500,-7.334999e+05
1,201904,274541,-3.200335e+06,0.0,0.000000,0,0,548571.43750,-548571.43750,0.0,548571.43750,-6.080734e+05
1,201905,459132,-3.104311e+06,0.0,0.000000,0,0,291428.56250,-291428.56250,0.0,291428.56250,-2.447571e+05
...,...,...,...,...,...,...,...,...,...,...,...,...
79539,202003,0,0.000000e+00,0.0,-191898.421875,0,0,0.00000,0.00000,0.0,120000.00000,-7.975236e+05
79539,202004,0,0.000000e+00,0.0,-192012.421875,0,0,0.00000,0.00000,0.0,120000.00000,-8.189417e+05
79539,202005,0,0.000000e+00,0.0,-550620.312500,0,0,0.00000,0.00000,0.0,120000.00000,-7.462784e+05


# TargetDf

In [26]:
targetdf.head()

,id,Producto-Tipo,Signo,Monto,Periodo
0,3,A-A,Positivo,234285.71875,201906
1,4,A-A,Positivo,362857.15625,201912
2,6,A-A,Positivo,220000.00000,201901
3,6,A-A,Positivo,220000.00000,201904
4,7,A-A,Positivo,220000.00000,201903


In [27]:
targetdf2 = targetdf.copy()
signo = targetdf2['Signo'].values
montoSigno = []
for i in signo:
    if i == 'Positivo':
        montoSigno.append(1)
    else:
        montoSigno.append(-1)
montoSigno = np.array(montoSigno)
monto = targetdf2['Monto'].values
targetdf2['MontoSigno'] = np.multiply(monto,montoSigno) # Multiplicación
targetdf2.drop(['Signo' , 'Monto'] , axis = 1 , inplace = True)
targetdf2.reset_index(drop = True , inplace = True)

In [28]:
targetdf2

,id,Producto-Tipo,Periodo,MontoSigno
0,3,A-A,201906,234285.71875
1,4,A-A,201912,362857.15625
2,6,A-A,201901,220000.00000
3,6,A-A,201904,220000.00000
4,7,A-A,201903,220000.00000
...,...,...,...,...
539626,79537,E-E,201907,124385.00000
539627,79537,E-E,201905,124392.00000
539628,79537,E-E,201901,124590.00000
539629,79539,E-E,201911,120714.00000


In [29]:
pivotTarget = pd.pivot_table(targetdf2 , values = 'MontoSigno' , index = ['id' , 'Periodo'] , columns = 'Producto-Tipo' , fill_value = 0 , aggfunc = np.sum)

In [30]:
pivotTargetdf = pivotTarget.copy()
pivotTargetdf.reset_index(inplace = True)

In [31]:
pivotTargetdfid = pivotTargetdf['id']
#pivotTargetdfperiod = pivotTargetdf['Periodo']
pivotTargetdf.drop(['id'] , axis = 1 , inplace = True)

In [32]:
pivotTargetdf

Producto-Tipo,Periodo,A-A,B-B,C-D,D-E,E-E
0,201902,0.0,0,0.000000,120951,0
1,201903,0.0,0,0.000000,120207,0
2,201904,0.0,0,0.000000,134176,0
3,201905,0.0,0,0.000000,121011,0
4,201906,0.0,0,0.000000,129190,0
...,...,...,...,...,...,...
448990,202003,0.0,0,192012.421875,0,0
448991,202004,0.0,0,192084.421875,0,0
448992,202005,0.0,0,192155.718750,0,0
448993,202006,0.0,0,551496.312500,0,0


In [33]:
for i in pivotTargetdf.columns.values:
    pivotTargetdf[i] = pivotTargetdf[i].apply(test)
pivotTargetdf['id'] = pivotTargetdfid

In [34]:
pivotTargetdf

Producto-Tipo,Periodo,A-A,B-B,C-D,D-E,E-E,id
0,1,0,0,0,1,0,2
1,1,0,0,0,1,0,2
2,1,0,0,0,1,0,2
3,1,0,0,0,1,0,2
4,1,0,0,0,1,0,2
...,...,...,...,...,...,...,...
448990,1,0,0,1,0,0,79539
448991,1,0,0,1,0,0,79539
448992,1,0,0,1,0,0,79539
448993,1,0,0,1,0,0,79539


# Juntando DF

In [35]:
pivotFinal.reset_index(inplace = True)

In [36]:
pivotFinal['Key'] = pivotFinal['id'].apply(str) + "-" + pivotFinal['Periodo'].apply(str)

In [37]:
pivotFinal

Producto-Tipo,id,Periodo,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,F-I,F-J,G-K,Key
0,1,201901,0,-2.946055e+06,0.0,0.000000,0,0,377142.84375,-377142.84375,0.0,377142.84375,-6.108959e+05,1-201901
1,1,201902,260048,-2.888311e+06,0.0,0.000000,0,0,0.00000,-120000.00000,0.0,120000.00000,-4.875886e+05,1-201902
2,1,201903,281602,-3.674344e+06,0.0,0.000000,0,0,577142.87500,-577142.87500,0.0,577142.87500,-7.334999e+05,1-201903
3,1,201904,274541,-3.200335e+06,0.0,0.000000,0,0,548571.43750,-548571.43750,0.0,548571.43750,-6.080734e+05,1-201904
4,1,201905,459132,-3.104311e+06,0.0,0.000000,0,0,291428.56250,-291428.56250,0.0,291428.56250,-2.447571e+05,1-201905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409148,79539,202003,0,0.000000e+00,0.0,-191898.421875,0,0,0.00000,0.00000,0.0,120000.00000,-7.975236e+05,79539-202003
1409149,79539,202004,0,0.000000e+00,0.0,-192012.421875,0,0,0.00000,0.00000,0.0,120000.00000,-8.189417e+05,79539-202004
1409150,79539,202005,0,0.000000e+00,0.0,-550620.312500,0,0,0.00000,0.00000,0.0,120000.00000,-7.462784e+05,79539-202005
1409151,79539,202006,0,0.000000e+00,0.0,-192155.718750,0,-120357,0.00000,0.00000,0.0,120000.00000,-1.204000e+06,79539-202006


In [38]:
pivotTargetdf['Key'] = pivotTargetdf['id'].apply(str) + "-" + pivotTargetdf['Periodo'].apply(str)

# Transacciones Final

In [40]:
final = pd.merge(pivotFinal,pivotTargetdf, on = 'Key' , how = 'left').fillna(0)
final

Producto-Tipo,id_x,Periodo_x,A-G,A-K,A-L,C-C,D-F,E-F,F-D,F-H,...,F-J,G-K,Key,Periodo_y,A-A,B-B,C-D,D-E,E-E,id_y
0,1,201901,0,-2.946055e+06,0.0,0.000000,0,0,377142.84375,-377142.84375,...,377142.84375,-6.108959e+05,1-201901,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,201902,260048,-2.888311e+06,0.0,0.000000,0,0,0.00000,-120000.00000,...,120000.00000,-4.875886e+05,1-201902,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,201903,281602,-3.674344e+06,0.0,0.000000,0,0,577142.87500,-577142.87500,...,577142.87500,-7.334999e+05,1-201903,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,201904,274541,-3.200335e+06,0.0,0.000000,0,0,548571.43750,-548571.43750,...,548571.43750,-6.080734e+05,1-201904,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,201905,459132,-3.104311e+06,0.0,0.000000,0,0,291428.56250,-291428.56250,...,291428.56250,-2.447571e+05,1-201905,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1409148,79539,202003,0,0.000000e+00,0.0,-191898.421875,0,0,0.00000,0.00000,...,120000.00000,-7.975236e+05,79539-202003,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1409149,79539,202004,0,0.000000e+00,0.0,-192012.421875,0,0,0.00000,0.00000,...,120000.00000,-8.189417e+05,79539-202004,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1409150,79539,202005,0,0.000000e+00,0.0,-550620.312500,0,0,0.00000,0.00000,...,120000.00000,-7.462784e+05,79539-202005,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1409151,79539,202006,0,0.000000e+00,0.0,-192155.718750,0,-120357,0.00000,0.00000,...,120000.00000,-1.204000e+06,79539-202006,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
final.to_pickle('TransaccionesTrainingSignosPeriodo.pkl')